In [1]:
import pymongo
from pprint import pprint

client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["mflix"]
my_collection = db["movies_initial"]


Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mflix')


In [3]:
pprint(client.mflix.movies_initial.find_one({"title": "The Big Short"}))


{'_id': ObjectId('6384c56d6a4fef4ccfbb6b6f'),
 'awards': '',
 'cast': 'Brad Pitt, Christian Bale, Selena Gomez, Karen Gillan',
 'country': 'USA',
 'director': 'Adam McKay',
 'fullplot': 'The men who made millions from a global economic meltdown.',
 'genre': 'Drama',
 'imdbID': 1596363,
 'imdbRating': '',
 'imdbVotes': '',
 'language': '',
 'lastupdated': '2015-09-12 00:20:34.633000000',
 'metacritic': '',
 'plot': 'The men who made millions from a global economic meltdown.',
 'poster': '',
 'rating': '',
 'released': '',
 'runtime': '',
 'title': 'The Big Short',
 'type': 'movie',
 'writer': 'Michael Lewis (based on the book by), Adam McKay (screenplay), '
           'Charles Randolph (screenplay)',
 'year': 2016}


In [ ]:
pipeline = [ {
   "$group": {
         "_id": "$language",
         # Count the number of movies in the group:
         "movie_count": { "$count": {} }, 
         
   }
}]

results = my_collection.aggregate(pipeline)
for type in results:
    pprint(type)


In [6]:
pipeline = [ 
    {
       '$match': {
           'language': {'$regex': "^Englisch"}
       } 
    },
    {
       "$group": {
             "_id": "$language",
             # Count the number of movies in the group:
             "movie_count": { "$count": {} }, 
         
   }
}]

results = my_collection.aggregate(pipeline)
for type in results:
    pprint(type)

In [ ]:
pipeline = [ 
    {
       "$group": {
             "_id": "$language",
             "movie_count": { "$sum": 1 },     
       }
    },  
    {
       "$sort": { "movie_count": pymongo.DESCENDING }
    }
]

results = my_collection.aggregate(pipeline)
for type in results:
    pprint(type)

In [ ]:
pipeline = [ 
    {
        '$sortByCount': "$language"
    }
]

results = my_collection.aggregate(pipeline)
for type in results:
    pprint(type)

In [10]:
cnt1Pipeline = [ { '$count': 'year' } ] 
pprint(list(my_collection.aggregate(cnt1Pipeline)))

[{'year': 46014}]


In [11]:
cnt2Pipeline = [ { '$count': 'director' } ] 
pprint(list(my_collection.aggregate(cnt2Pipeline)))

[{'director': 46014}]


In [12]:
cnt3Pipeline = [ { '$count': 'writer' } ] 
pprint(list(my_collection.aggregate(cnt3Pipeline)))

[{'writer': 46014}]


In [19]:
pipeline = [ 
    
    {
        '$group': {
          '_id': '1',
          'max': { '$max': '$year' },
          'min': { '$min': '$year'}  
        },
    }
]

results = my_collection.aggregate(pipeline)
for type in results:
    pprint(type)


{'_id': '1', 'max': '2016�', 'min': 1874}


In [ ]:
import pymongo
from pprint import pprint

client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["mflix"]
my_collection = db["movies_initial"]

pipelineFindAll = [ 
    {
        '$project': {
          '_id': 1, 
          'year': 1
        }
    } 
]

results = list(my_collection.aggregate(pipelineFindAll))
for i in results:
    print(i)

In [9]:
from bson.objectid import ObjectId

In [16]:
#for e in results:
 #   if isinstance(e["year"], str):
  #      updateStringID = { '_id': e["_id"] }
   #     updateStringDate = { '$set': { 'year': int(e["year"][0:4])}}
    #    pprint(f"Update {e['year']} -> {updateStringID}, {updateStringDate}")
     #   my_collection.update_one(updateStringID, updateStringDate)

KeyError: 'year'

In [15]:
pipeline = [ 
    
    {
        '$group': {
          '_id': '1',
          'max': { '$max': '$year' },
          'min': { '$min': '$year'}  
        },
    }
]

results = list(my_collection.aggregate(pipeline))
for type in results:
    pprint(type)

{'_id': '1', 'max': 2019, 'min': 1874}


In [18]:
maxYear = int(results[0]["max"])
minYear = int(results[0]["min"])
print(maxYear)
print(minYear)

2019
1874


In [ ]:
pipeline=[
    # die match-Klausel ist fuer Aufgabe 1b - nur Sprachkombis, die mit "English" anfangen
    {
        '$match': {
            'language': { '$regex': "^English" }
        }
    },
    {
        '$group': {
            '_id': "$language",
            'cnt': { '$sum': 1 },
            # 'cntcnt': { '$count': {}}
        }
    }
]

pprint(list(my_collection.aggregate(pipeline)))

In [25]:
pipelineYear2016=[
    {
        '$match': {
            'year': 2016
        }
    },
    
    #braucht nicht zwingend ein group, hätte auch einfach nur count direkt machen können -> in der Praxis eher nutzen
    {
       "$group": {
             "_id": "$year",
             # Count the number of movies in the group:
             "movie_count": { "$count": {} } 
         
       }
    }
]

pprint(list(my_collection.aggregate(pipelineYear2016)))

[{'_id': 2016, 'movie_count': 523}]


In [27]:
print(minYear)
print(maxYear)

1874
2019
[{'_id': '1', 'max': 2019, 'min': 1874}]


In [35]:
pipelineYears=[
    {
        "$group": {
            "_id": "$year"
        }
    },
    {
       "$sort": { "_id": pymongo.ASCENDING }
    }
]

years = list(my_collection.aggregate(pipelineYears))
print(years[0]['_id'])


1874


In [46]:
import json
import datetime

start = datetime.datetime.now()
i = 0
for n in years:
    query = '{"year": ' + str(years[i]['_id']) + '}'
    query = json.loads(query)
    i=i+1
    find = my_collection.find(query)
end = datetime.datetime.now() 
diff = (end - start).total_seconds()
print(diff)

0.001995


In [50]:
start = datetime.datetime.now()

i = 0
for n in years:
    pipelineFindPerYear = [
        {
            "$match": {
                'year': years[0]['_id']
            }
        }
    ]

    resultsPerYear = my_collection.aggregate(pipelineFindPerYear)

end = datetime.datetime.now() 
diff = (end - start).total_seconds()
print(diff)

3.785547


In [48]:
jahresliste = my_collection.distinct('year')
print(jahresliste)

[1874, 1880, 1887, 1888, 1890, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


In [49]:
# find lazy, aggregation pipeline nicht. bei find iterator und liefert nicht alle ergebnisse gleichzeitig, bei list wird alles daraus genommen bis es nichts mehr gibt, ohne list das erste ergebnis oder keins. "Lazy evaluation" (kann sehr gut in der klausur vorkommen, vlt lieber lazy den find als agg. pipeline)

# Aufgabe 8
finde alle Filme von 2016 und gruppiere sie nach rating zusammen mit der Anzahl der jeweiligen Art von rating, aufsteigend sortiert


In [62]:
pipeline8=[
    {
        '$match': {
            'year': 2016
        } 
    },
    {
        '$group': {
            '_id': '$rating',
            'rating_count': { '$count': {} }
       }
    },
    {
        '$sort': { "rating_count": 1 }
    }
#    {
 #       '$sortByCount': "$rating"
  #  }
]

results8 = my_collection.aggregate(pipeline8)
for r in results8:
    pprint(r)

{'_id': 'PG-13', 'rating_count': 1}
{'_id': 'R', 'rating_count': 5}
{'_id': '', 'rating_count': 517}


# Aufgabe 9
finde alle Short Filme die seit 2000 produziert wurden:

- wie viele waren es?
- liste jeweils den Titel und das Jahr, alphabetisch nach Titel sortiert
- die resultierenden Daten sollen in der DB ShortMovies in der Collection 2000er stehen

In [85]:
pipeline9 = [
    {
        '$match': {
            'year': { '$gt': 1999 },
            'genre': { '$regex': "Short" }
        }
    },
    {
        '$count': "genre"
    }
]

results9 = my_collection.aggregate(pipeline9)
for res in results9:
    pprint(f" a) {res}")

" a) {'genre': 559}"


In [ ]:
pipeline9b = [
    {
        '$match': {
            'year': { '$gt': 1999 },
            'genre': { '$regex': ".*Short.*" }
        }
    },
    {
        '$project': {
            'year': 1,
            'title': 1
        }   
    },
    {
        '$group': {
            '_id': '$title',
            'year': { '$first': '$year'}
        }
    },
    {
       "$sort": { '_id': pymongo.ASCENDING }
    }
]

results9b = list(my_collection.aggregate(pipeline9b))
for res in results9b:
    pprint(res)

In [90]:
pipeline9c = [
    {
        '$match': {
            'year': { '$gt': 1999 },
            'genre': { '$regex': ".*Short.*" }
        }
    },
    {
        '$project': {
            'year': 1,
            'title': 1
        }   
    },
    {
        '$group': {
            '_id': '$title',
            'year': { '$first': '$year'}
        }
    },
    {
       "$sort": { '_id': pymongo.ASCENDING }
    },
    {
        '$out': { 'db': 'ShortMovies', 'coll': '2000er'}
    }
]

res9c = my_collection.aggregate(pipeline9c)